In [ ]:
# !pip install -qU  langchain_milvus python-dotenv langchain-openai langchain_ollama langchain_community GitPython

In [39]:
import os
import shutil
import time
import logging
from dotenv import load_dotenv
from git import Repo
from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader
from openai import OpenAI

In [ ]:
# import os
# import shutil
# import numpy as np
# import pandas as pd
# from dotenv import load_dotenv
# from git import Repo
# from langchain_milvus import Milvus
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.document_loaders import GitLoader
# from openai import OpenAI

# class GitHubGPT:
#     def __init__(self):
#         self.OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#         self.embeddings = self.__initialize_embeddings()
#         self.vector_db = self.__initialize_vector_db()
#         self.client = OpenAI(api_key=self.OPENAI_API_KEY)
#         self.system_prompt = self.__initialize_system_prompt()
#         self.thread = None
#         self.thread_id = None
#         self.messages = []  # List to store messages

#     def __initialize_embeddings(self):
#         return OpenAIEmbeddings(
#             model="text-embedding-3-small",
#             openai_api_key=self.OPENAI_API_KEY
#         )

#     def __initialize_vector_db(self):
#         if not os.path.exists("./vector_db"):
#             os.makedirs("./vector_db", mode=0o777)
            
#         return Milvus(
#             embedding_function=self.embeddings,
#             connection_args={"uri": "./vector_db/milvus_example.db"},
#             auto_id=True,
#             collection_name="github_gpt",
#         )
        
#     def __initialize_system_prompt(self):
#         return '''
#     What are you? A well informed, intelligent chatbot which can talk to a given codebase.
#     What do you do? You are always given some file content from a codebase and a question/prompt. Your job is to generate a response.
#     What should be the tone of your output? It should be friendly, helpful, confident, narrative.
#     What outputs can we expect from you? You can be asked to generate documentations, code, or anything else only relevant to the given codebase content.
#     '''

#     @staticmethod
#     def __clean_repo_name(name):
#         return name.replace('-', '_')
    
#     @staticmethod
#     def __declean_repo_name(name):
#         return name.replace('_', '-')
    
#     def __add_repo_data_to_db(self):
#         data = self.loader.load()
#         print(f'Length of Data to Add: {len(data)}')
#         print(f'Adding Data to Milvus Vector DB')
#         self.vector_db.add_documents(documents=data)
#         print(f'Done Adding Data to Milvus Vector DB')
    
#     def add_repo(self, repo_url):
#         repo_name = repo_url.split('/')[-1]
#         repo_save_path = f"./Data/Repos"
#         if not os.path.exists(repo_save_path):
#             os.makedirs(repo_save_path)
#         else:
#             shutil.rmtree(repo_save_path)
#             os.makedirs(repo_save_path)
#         repo_save_path = repo_save_path + "/" + self.__clean_repo_name(repo_name)
        
#         print(f'Cloning the repo from: {repo_url}')
#         repo = Repo.clone_from(
#             repo_url, 
#             to_path=repo_save_path,
#             branch="master"
#         )
#         print(f'Repo Cloned to: {repo_save_path}')
#         self.repo_save_path = repo_save_path
#         self.branch = repo.head.reference
#         self.loader = GitLoader(repo_path=repo_save_path, branch=self.branch)
#         self.__add_repo_data_to_db()

#     def load_repo(self):
#         repo_save_path = "./Data/Repos"
#         repo_name = os.listdir(repo_save_path)[0]
#         self.repo_save_path = repo_save_path + "/" + repo_name
#         self.branch = "master"
#         print(f'Loading repo: {repo_name}')
#         print(f'Branch: {self.branch}')
#         print(f'Repo path: {self.repo_save_path}')
#         self.loader = GitLoader(repo_path=self.repo_save_path, branch=self.branch)
#         self.__add_repo_data_to_db()

#     def __retrieve_documents(self, prompt, k=3):
#         retrieved_documents = self.vector_db.similarity_search(
#             prompt,
#             k=k
#         )
#         return retrieved_documents
    
#     @staticmethod
#     def __concatenate_documents(documents):
#         print(f'Length of docs to concatenate: {len(documents)}')
#         All_content = ''
#         for idx, doc in enumerate(documents):
#             print(f"Retrieved Document: {idx} --- [{doc.metadata}]")
#             All_content += "Chunk:" + str(idx) + ":\n" + doc.page_content + "\n\n"
#         print("\n\n")
#         return All_content
    
#     def query(self, prompt):
#         retrieved_documents = self.__retrieve_documents(prompt)
#         context = self.__concatenate_documents(retrieved_documents)

#         # Add the user message to the messages list
#         self.messages.append({"role": "user", "content": f"Context from codebase:{context}\nUser query prompt:{prompt}\nResponse:\n"})
        
#         completion = self.client.chat.completions.create(
#         model="gpt-4o",
#         messages=[
#             {"role": "system", "content": f"{self.system_prompt}"},
#             {"role": "user", "content": f"Context from codebase:{context}\nUser query prompt:{prompt}\nResponse:\n"},
#         ],
#         stream=True
#         )
        
#         response_text = ''
#         for chunk in completion:
#             if chunk.choices[0].delta.content is not None:
#                 response_text += chunk.choices[0].delta.content
#                 yield chunk.choices[0].delta.content
        
#         self.messages.append({"role": "assistant", "content": response_text})
#         print(f'\n\nMessage has been added to the list of messages: {self.messages}, which will be later on a part of the retrieved thread. To retrieve the thread, call retrieve_thread()')
    
#     def retrieve_thread(self):
#         print(f'self.messages:')
#         print(self.messages)
#         thread = self.client.beta.threads.create(messages=self.messages)
#         self.thread_id = thread.id
#         self.thread = thread
#         print(f'The thread ID is: {thread.id}')
#         return thread

In [49]:
class GitHubGPT:
    def __init__(self):
        self.OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        self.embeddings = self.__initialize_embeddings()
        self.vector_db = self.__initialize_vector_db()
        self.client = OpenAI(api_key=self.OPENAI_API_KEY)
        self.system_prompt = self.__initialize_system_prompt()
        self.thread_id = None
        self.assistant_id = self.__create_assistant(name='Github GPT', instructions='Please address the user as Github GPT')

    def __initialize_embeddings(self):
        return OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=self.OPENAI_API_KEY
        )

    def __initialize_vector_db(self):
        if not os.path.exists("./vector_db"):
            os.makedirs("./vector_db", mode=0o777)
            
        return Milvus(
            embedding_function=self.embeddings,
            connection_args={"uri": "./vector_db/milvus_example.db"},
            auto_id=True,
            collection_name="github_gpt",
        )
        
    def __initialize_system_prompt(self):
        return '''
    What are you? A well-informed, intelligent chatbot that can interact with a codebase.
    What do you do? You are always provided with some file content from a codebase and a question/prompt. Your job is to generate a response.
    What should be the tone of your output? It should be friendly, helpful, confident, and narrative.
    What outputs can we expect from you? You can be asked to generate documentations, code, or anything else only relevant to the given codebase content.
    '''
    
    @staticmethod
    def __clean_repo_name(name):
        return name.replace('-', '_')

    def __create_assistant(self, name, instructions, model="gpt-3.5-turbo-16k"):
        assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            model=model,
        )
        print(f'Assistant created with ID: {assistant.id}')
        return assistant.id

    def __retrieve_documents(self, prompt, k=3):
        retrieved_documents = self.vector_db.similarity_search(
            prompt,
            k=k
        )
        return retrieved_documents
    
    @staticmethod
    def __concatenate_documents(documents):
        print(f'Length of docs to concatenate: {len(documents)}')
        All_content = ''
        for idx, doc in enumerate(documents):
            print(f"Retrieved Document: {idx} --- [{doc.metadata}]")
            All_content += "Chunk:" + str(idx) + ":\n" + doc.page_content + "\n\n"
        print("\n\n")
        return All_content

    def query(self, prompt, instructions="Please address the user as Github User"):
        # Step 1: Retrieve relevant documents based on the user's query
        retrieved_documents = self.__retrieve_documents(prompt)
        context = self.__concatenate_documents(retrieved_documents)

        # Step 2: Create a thread using the retrieved context and user prompt
        user_query = f"Context from codebase: {context}\nUser query: {prompt}\n"
        thread = self.client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": user_query,
                }
            ]
        )
        self.thread_id = thread.id
        print(f'Thread created with ID: {self.thread_id}')

        # Step 3: Run the assistant on the created thread
        run = self.client.beta.threads.runs.create(
            thread_id=self.thread_id,
            assistant_id=self.assistant_id,
            instructions=instructions,
        )

        # Step 4: Wait for the assistant's response
        self.wait_for_run_completion(self.client, thread_id=self.thread_id, run_id=run.id)

    def wait_for_run_completion(self, client, thread_id, run_id, sleep_interval=5):
        while True:
            try:
                run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
                if run.completed_at:
                    elapsed_time = run.completed_at - run.created_at
                    formatted_elapsed_time = time.strftime(
                        "%H:%M:%S", time.gmtime(elapsed_time)
                    )
                    print(f"Run completed in {formatted_elapsed_time}")
                    logging.info(f"Run completed in {formatted_elapsed_time}")
                    # Get messages here once the run is completed!
                    messages = client.beta.threads.messages.list(thread_id=thread_id)
                    print(f'All messages: {messages.data}\n')
                    last_message = messages.data[0]
                    response = last_message.content[0].text.value
                    print(f"Assistant Response: {response}\n\n****\n\n")
                    break
            except Exception as e:
                logging.error(f"An error occurred while retrieving the run: {e}")
                break
            logging.info("Waiting for run to complete...")
            time.sleep(sleep_interval)


In [50]:
obj = GitHubGPT()

Assistant created with ID: asst_s1Mo1hXQcPH91c23YQTHbVcr


### Below functions / statements are responsile to 
- clone + load the data into the vectro db
- load the already cloned data into the vector db
Hence only uncomment one which you want to use, else the data will be already in the local vector db.

In [ ]:
# obj.add_repo("https://github.com/SaschaNe/creatify-app")
# obj.load_repo()

In [51]:
res = obj.query("Explain the implementation of the processPersonProfile function in the CrispService class.")
res
# for chunk in res:
#     print(chunk, end='', flush=True)

Length of docs to concatenate: 3
Retrieved Document: 0 --- [{'file_name': 'crispChat.js', 'file_path': 'resources/js/crispChat.js', 'file_type': '.js', 'pk': 452165072252045620, 'source': 'resources/js/crispChat.js'}]
Retrieved Document: 1 --- [{'file_name': 'CrispService.php', 'file_path': 'app/Services/CrispService.php', 'file_type': '.php', 'pk': 452165072252045403, 'source': 'app/Services/CrispService.php'}]
Retrieved Document: 2 --- [{'file_name': 'CrispPeople.php', 'file_path': 'app/Models/CrispPeople.php', 'file_type': '.php', 'pk': 452165072252045330, 'source': 'app/Models/CrispPeople.php'}]



Thread created with ID: thread_iemGU2VY3SfjVVB36O9NpQoM
Run completed in 00:00:07
All messages: [Message(id='msg_xUw9dhJ6MJ0BQA0zleJ4yjIh', assistant_id='asst_s1Mo1hXQcPH91c23YQTHbVcr', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The processPersonProfile function in the CrispService class is responsible for handling and processing the pro

TypeError: 'NoneType' object is not iterable

In [ ]:
res = obj.query("List all the middlewares which are not included in the standard Laravel 10 application and custom developed.")
for chunk in res:
    print(chunk, end='', flush=True)

In [ ]:
id = obj.retrieve_thread()

In [ ]:
# import openai
# from dotenv import find_dotenv, load_dotenv
# import time
# import logging
# from datetime import datetime

# load_dotenv()
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# # defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# # if you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )

# client = openai.OpenAI()
# model = "gpt-3.5-turbo-16k"

# # ==  Create our Assistant (Uncomment this to create your assistant) ==
# personal_trainer_assis = client.beta.assistants.create(
#     name="Personal Trainer",
#     instructions="""You are the best personal trainer and nutritionist who knows how to get clients to build lean muscles.\n
#      You've trained high-caliber athletes and movie stars. """,
#     model=model,
# )
# asistant_id = personal_trainer_assis.id
# print(asistant_id)


# # === Thread (uncomment this to create your Thread) ===
# thread = client.beta.threads.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "How do I get started working out to lose fat and build muscles?",
#         }
#     ]
# )
# thread_id = thread.id
# print(thread_id)

# # # === Hardcode our ids ===
# # asistant_id = "asst_7yT5g1MShqrN1534vDrqm810"
# # thread_id = "thread_kr871DUEaGcTXkBb2jHO9eJ6"

# # === Run our Assistant ===
# run = client.beta.threads.runs.create(
#     thread_id=thread_id,
#     assistant_id=asistant_id,
#     instructions="Please address the user as James Bond",
# )

# def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
#     """

#     Waits for a run to complete and prints the elapsed time.:param client: The OpenAI client object.
#     :param thread_id: The ID of the thread.
#     :param run_id: The ID of the run.
#     :param sleep_interval: Time in seconds to wait between checks.
#     """
#     while True:
#         try:
#             run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
#             if run.completed_at:
#                 elapsed_time = run.completed_at - run.created_at
#                 formatted_elapsed_time = time.strftime(
#                     "%H:%M:%S", time.gmtime(elapsed_time)
#                 )
#                 print(f"Run completed in {formatted_elapsed_time}")
#                 logging.info(f"Run completed in {formatted_elapsed_time}")
#                 # Get messages here once Run is completed!
#                 messages = client.beta.threads.messages.list(thread_id=thread_id)
#                 last_message = messages.data[0]
#                 response = last_message.content[0].text.value
#                 print(f"Assistant Response: {response}")
#                 break
#         except Exception as e:
#             logging.error(f"An error occurred while retrieving the run: {e}")
#             break
#         logging.info("Waiting for run to complete...")
#         time.sleep(sleep_interval)


# # === Run ===
# wait_for_run_completion(client=client, thread_id=thread_id, run_id=run.id)

# # ==== Steps --- Logs ==
# run_steps = client.beta.threads.runs.steps.list(thread_id=thread_id, run_id=run.id)
# print(f"Steps---> {run_steps.data[0]}")